In [ ]:
import sys
import os
import json
import pandas as pd 
import numpy as np
import os,glob
import re
from pathlib import Path
import pandas as pd 
import numpy as np
from collections import defaultdict 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

sys.path.insert(0, '../src')

In [ ]:
with open('../config/data-params.json') as fh:
            data_cfg = json.load(fh)
from build_features import create_struct
data_dict = create_struct(data_cfg['PATH'])


In [ ]:
def_dict = data_dict

## Feature Engineering

In [ ]:
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []
col7 = []
col8 = []

for elem1 in list(def_dict.keys()):
    for elem2 in list(def_dict[elem1].keys()):
        col1.append(elem1)
        col2.append(elem2)
        col3.append(len(def_dict[elem1][elem2]['combined']['APIs']))
        col4.append(len(def_dict[elem1][elem2]['invoke_type']['invoke-static']))
        col5.append(len(def_dict[elem1][elem2]['invoke_type']['invoke-virtual']))
        col6.append(len(def_dict[elem1][elem2]['invoke_type']['invoke-direct']))
        col7.append(len(def_dict[elem1][elem2]['invoke_type']['invoke-super']))
        col8.append(len(def_dict[elem1][elem2]['invoke_type']['invoke-interface']))
df = pd.DataFrame([col1, col2, col3, col4, col5, col6, col7, col8]).T
df['App Category'] = df[0]
df['All API'] = df[2]
df['invoke-static'] = df[3]
df['invoke-virtual'] = df[4]
df['invoke-direct'] = df[5]
df['invoke-super'] = df[6]
df['invoke-interface'] = df[7]
df = df.drop([0,2,3,4,5,6,7],1)     

In [ ]:
df.head()

## Visual Distribution Analysis

In [ ]:
df[['App Category', 'All API']].sort_values(by = 'App Category').plot(kind= 'hist', figsize = [10,6])

In [ ]:
df.loc[df['App Category'] == 'malware'].plot(figsize = [10,5])

In [ ]:
df.loc[df['App Category'] == 'benign'].plot(figsize = [10,5])

## Understanding Features

In [ ]:
df.head()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.violinplot(data=pd.melt(df), x=1, y='All API', ax=axes[0])

In [ ]:
df.groupby('App Category').median()

In [ ]:
df.groupby('App Category').mean()

In [ ]:
set(def_dict['benign']['combined']['APIs'])

In [ ]:
api_final = []
for category in def_dict:
    for appName in def_dict[category]:
        print(appName)
        api_final.append(list(def_dict[category][appName]['combined']['APIs']))

A_matrix = {}
for index, elem in enumerate(allAPIs):
    A_matrix[index] = set(elem)



In [ ]:
for index in A_matrix:
    pd.Series(set(A_matrix[index])).groupby('index').count().plot(kind = 'hist', bins = [0,100,1])

In [ ]:
from scipy.stats import normaltest

df.apply(lambda x: pd.Series(normaltest(x), index=['skew-test + kurtosis-test', 'p-value'])).T
    

## Baseline Model 

In [ ]:
y = df['App Category'].astype('category').cat.codes
X = df.drop(['App Category',1], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
grandientboosting = GradientBoostingClassifier()
grandientboosting.fit(X_train,y_train)
grandientboosting.predict(X_test)
grandientboosting.score(X_test, y_test)